In [1]:
%cd drive/MyDrive/Colab Notebooks/big data sample

/content/drive/MyDrive/Colab Notebooks/big data sample


In [17]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import os

pd.options.mode.chained_assignment = None

In [25]:
path = 'weather prediction/'

if not os.path.isfile(path + 'district.csv'):
        ###前3列為標題和日期等無用資料
        df = pd.read_excel('https://www.stat.gov.tw/public/Attachment/712693030RPKUP4RX.xlsx', header=3)
        ###去除不相關的內容，inplace=True會更新原資料的內容
        df.drop(columns=['縣市代碼', '村里代碼', '村里名稱', '村里代碼', '村里代碼.1'], axis=1, inplace=True)
        ###移除掉重複的資料
        df.drop_duplicates(inplace=True)
        df.to_csv(path + 'district.csv', encoding='big5', index=False)

In [6]:
town =pd.read_csv(path+'district.csv', encoding='big5')
area = input('請輸入縣市(使用\'台\')及地區:')

請輸入縣市(使用'台')及地區:台北市大同區


In [7]:
dfs = town[( town['縣市名稱'] == area[:3]) & ( town['區鄉鎮名稱']==area[3:])]

if len(dfs) > 0 :
  output = str(dfs.iloc[0, 1])
  print(output)
else:
  print('輸入資訊找不到或是有錯誤')       

6300600


In [8]:
url = 'https://www.cwb.gov.tw/V8/C/W/Town/MOD/3hr/'+ output +'_3hr_PC.html'
html = requests.get(url)
html.encoding = 'utf-8'
soup = BeautifulSoup(html.text, 'lxml')


###資料會有很多''，把他們全部去掉
data = soup.text.split('\n')
print('原本資料長度:' + str(len(data)))

count = 0
for i in range(len(data)):
   if data[i] == '':
     count += 1
print('共有{}個位置是'''.format(count))

for num in range(count):
   data.remove('')
print('資料長度變:', str(len(data)))

#data

原本資料長度:203
共有43個位置是
資料長度變: 160


In [9]:
###改變日期的形式:星期六 -> (六)
date = data[2:5]
for i in range(len(date)):
  date[i] = date[i][:-3] + '(' + date[i][-1] + ')'

###去掉沒有每個時刻都會回報的數據
del data[0:5]

###天氣情況存到img中，另外抓取
data.remove('天氣狀況')

len(data)

154

In [10]:
###找到天氣情況存在img標籤中的alt中
weather = []
for img in soup.find_all('img'):
  weather.append(img.get('alt'))    

In [11]:
###因為降雨機率是每兩個時刻報一次(除了最後一個時刻外)，因此為了補齊和其他種類的長度，人工補齊
index1 = data.index('降雨機率')
ind = index1+1
for num in range(8):
   value = data[ind]
   data.insert(ind, value)
   ind += 2

print(len(data))
print(data[index1:index1+18])
print(data[index1:index1+19])

162
['降雨機率', '20%', '20%', '30%', '30%', '40%', '40%', '60%', '60%', '80%', '80%', '90%', '90%', '90%', '90%', '80%', '80%', '80%']
['降雨機率', '20%', '20%', '30%', '30%', '40%', '40%', '60%', '60%', '80%', '80%', '90%', '90%', '90%', '90%', '80%', '80%', '80%', '相對濕度']


In [12]:
###把時間，溫度，風速等等資訊以每17個17個抓出來，存成pd，再合併起來

info = []
name = []
num = 0
for num in range(9):
  name.append(data[num*18])
  time = data[num*18+1:(num+1)*18]
  ###溫度方面會顯示到四位數，為了和網站上相同單位，取前兩位即可
  if data[num*18] == '溫度' or data[num*18] == '體感溫度':
    time = [i[:2] for i in time]
  print(time)
  df = pd.DataFrame(time)
  info.append(df)

###axis=1，按照column一列一列接續
df1 = pd.concat(info, axis=1, ignore_index=True)
df1.columns = name
df1.insert(1, '日期', 0)

['21:00', '00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00', '00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00']
['22', '22', '22', '21', '21', '21', '21', '21', '21', '21', '20', '20', '20', '20', '20', '19', '19']
['25', '25', '25', '24', '24', '23', '22', '23', '23', '23', '23', '22', '22', '22', '21', '20', '19']
['20%', '20%', '30%', '30%', '40%', '40%', '60%', '60%', '80%', '80%', '90%', '90%', '90%', '90%', '80%', '80%', '80%']
['94%', '90%', '88%', '87%', '86%', '83%', '81%', '85%', '86%', '87%', '88%', '86%', '84%', '85%', '84%', '83%', '85%']
['≤1', '≤1', '≤1', '≤1', '≤1', '2', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '≤1', '2', '2', '2', '≤1']
['1', '0', '1', '1', '2', '2', '2', '1', '1', '1', '2', '1', '1', '2', '2', '2', '2']
['偏西風', '西北風', '偏北風', '東北風', '偏北風', '偏北風', '偏北風', '西北風', '西北風', '偏北風', '東北風', '偏北風', '偏北風', '偏北風', '東北風', '東北風', '東北風']
['舒適', '舒適', '舒適', '舒適', '舒適', '舒適', '舒適', '舒適', '舒適', '舒適', '稍有寒意', '稍有寒意', '稍有寒意', '稍有寒意', '稍

In [13]:
###把日期加上去
j=0
for i in range(len(df1.index)):
  ###遇到0點要換一天，除了第一個外
  if i==0 and df1.iloc[i, 0] == '00:00':
    pass
  elif df1.iloc[i, 0] == '00:00':
    j += 1  
  df1.iloc[i, 1] = date[j]

df1

,時間,日期,溫度,體感溫度,降雨機率,相對濕度,蒲福風級,風速(m/s),風向,舒適度
0,21:00,12/03(六),22,25,20%,94%,≤1,1,偏西風,舒適
1,00:00,12/04(日),22,25,20%,90%,≤1,0,西北風,舒適
2,03:00,12/04(日),22,25,30%,88%,≤1,1,偏北風,舒適
3,06:00,12/04(日),21,24,30%,87%,≤1,1,東北風,舒適
4,09:00,12/04(日),21,24,40%,86%,≤1,2,偏北風,舒適
5,12:00,12/04(日),21,23,40%,83%,2,2,偏北風,舒適
6,15:00,12/04(日),21,22,60%,81%,≤1,2,偏北風,舒適
7,18:00,12/04(日),21,23,60%,85%,≤1,1,西北風,舒適
8,21:00,12/04(日),21,23,80%,86%,≤1,1,西北風,舒適
9,00:00,12/05(一),21,23,80%,87%,≤1,1,偏北風,舒適


In [14]:
###把天氣情況一個一個加上去
df1.insert(2, '天氣情況', '!')
for i in range(len(weather)):
  df1.iloc[i, 2] = weather[i]

df1

,時間,日期,天氣情況,溫度,體感溫度,降雨機率,相對濕度,蒲福風級,風速(m/s),風向,舒適度
0,21:00,12/03(六),晴,22,25,20%,94%,≤1,1,偏西風,舒適
1,00:00,12/04(日),短暫雨,22,25,20%,90%,≤1,0,西北風,舒適
2,03:00,12/04(日),短暫雨,22,25,30%,88%,≤1,1,偏北風,舒適
3,06:00,12/04(日),短暫雨,21,24,30%,87%,≤1,1,東北風,舒適
4,09:00,12/04(日),短暫雨,21,24,40%,86%,≤1,2,偏北風,舒適
5,12:00,12/04(日),短暫雨,21,23,40%,83%,2,2,偏北風,舒適
6,15:00,12/04(日),短暫雨,21,22,60%,81%,≤1,2,偏北風,舒適
7,18:00,12/04(日),短暫雨,21,23,60%,85%,≤1,1,西北風,舒適
8,21:00,12/04(日),短暫雨,21,23,80%,86%,≤1,1,西北風,舒適
9,00:00,12/05(一),短暫雨,21,23,80%,87%,≤1,1,偏北風,舒適


In [15]:
###要使用Web API (如:Flask)時，通常參數是以json檔格式回傳
print(df1.to_json(orient='records', force_ascii=False))

[{"時間":"21:00","日期":"12\/03(六)","天氣情況":"晴","溫度":"22","體感溫度":"25","降雨機率":"20%","相對濕度":"94%","蒲福風級":"≤1","風速(m\/s)":"1","風向":"偏西風","舒適度":"舒適"},{"時間":"00:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"22","體感溫度":"25","降雨機率":"20%","相對濕度":"90%","蒲福風級":"≤1","風速(m\/s)":"0","風向":"西北風","舒適度":"舒適"},{"時間":"03:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"22","體感溫度":"25","降雨機率":"30%","相對濕度":"88%","蒲福風級":"≤1","風速(m\/s)":"1","風向":"偏北風","舒適度":"舒適"},{"時間":"06:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"21","體感溫度":"24","降雨機率":"30%","相對濕度":"87%","蒲福風級":"≤1","風速(m\/s)":"1","風向":"東北風","舒適度":"舒適"},{"時間":"09:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"21","體感溫度":"24","降雨機率":"40%","相對濕度":"86%","蒲福風級":"≤1","風速(m\/s)":"2","風向":"偏北風","舒適度":"舒適"},{"時間":"12:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"21","體感溫度":"23","降雨機率":"40%","相對濕度":"83%","蒲福風級":"2","風速(m\/s)":"2","風向":"偏北風","舒適度":"舒適"},{"時間":"15:00","日期":"12\/04(日)","天氣情況":"短暫雨","溫度":"21","體感溫度":"22","降雨機率":"60%","相對濕度":"81%","蒲福風級":"≤1","風速(m\/s)":"2","風向":"偏北風","舒適度":"舒適"},{"時間":"1